In [1]:
# Load data
import pandas as pd
import numpy as np
url = "SMSSpamCollection.txt"
df = pd.read_csv(url, sep='\t', header=None )
df.rename(columns={0:'type',1:'mail'}, inplace=True)
df


,type,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [2]:
df.describe()

,type,mail
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [3]:
df_ham=df[df['type']=='ham']

df_ham['split']=df_ham['mail'].str.split()
df_ham

C:\Users\MOHAM\AppData\Local\Temp\ipykernel_10916\1565084022.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ham['split']=df_ham['mail'].str.split()


,type,mail,split
0,ham,"Go until jurong point, crazy.. Available only ...","[Go, until, jurong, point,, crazy.., Available..."
1,ham,Ok lar... Joking wif u oni...,"[Ok, lar..., Joking, wif, u, oni...]"
3,ham,U dun say so early hor... U c already then say...,"[U, dun, say, so, early, hor..., U, c, already..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[Nah, I, don't, think, he, goes, to, usf,, he,..."
6,ham,Even my brother is not like to speak with me. ...,"[Even, my, brother, is, not, like, to, speak, ..."
...,...,...,...
5565,ham,Huh y lei...,"[Huh, y, lei...]"
5568,ham,Will ü b going to esplanade fr home?,"[Will, ü, b, going, to, esplanade, fr, home?]"
5569,ham,"Pity, * was in mood for that. So...any other s...","[Pity,, *, was, in, mood, for, that., So...any..."
5570,ham,The guy did some bitching but I acted like i'd...,"[The, guy, did, some, bitching, but, I, acted,..."


In [4]:
df_spam=df[df['type']=='spam']
df_spam

,type,mail
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [5]:
# data
import pandas as pd

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler

# Pipeline and model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
# Score of models
from sklearn.metrics import accuracy_score

# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, RobustScaler
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer

In [75]:
column_text=df.columns.drop(['type'])
column_text

Index(['texto'], dtype='object')

In [31]:
vectorizer2 = CountVectorizer(analyzer='word')
X2 = vectorizer2.fit_transform(X_train)
X2.shape
#X2.toarray()
#vectorizer2.get_feature_names_out()


(3900, 7120)

In [7]:
# Train test split

X = df['mail']
y = df['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: mail, Length: 5572, dtype: object

In [8]:
column_text=df.select_dtypes(include=['object']).columns.drop(['type'])
column_text

Index(['mail'], dtype='object')

In [9]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(X_train)
#X2.toarray()
vectorizer2.get_feature_names_out()

array(['00 in', '00 per', '00 sub', ..., 'zyada kisi', 'ú1 20',
       '〨ud evening'], dtype=object)

In [10]:
# Transformation of textual variables
transfo_text = Pipeline(steps=[
    ('bow', CountVectorizer(decode_error='ignore', analyzer='word', ngram_range=(2, 2)))
])

# Class ColumnTransformer : apply alls steps on the whole dataset
preparation = ColumnTransformer(
    transformers=[
        ('data_bow', transfo_text , column_text)
    ])
preparation

ColumnTransformer(transformers=[('data_bow',
                                 Pipeline(steps=[('bow',
                                                  CountVectorizer(decode_error='ignore',
                                                                  ngram_range=(2,
                                                                               2)))]),
                                 Index(['mail'], dtype='object'))])

In [11]:
# Creation of model : a ready to use pipeline for ML process

classifier =ComplementNB()
# classifier = MultinomialNB()


In [12]:
model_lm = Pipeline([
    ('vectorizer', transfo_text),
    ('classifier', classifier)
])

In [13]:
model_lm

Pipeline(steps=[('vectorizer',
                 Pipeline(steps=[('bow',
                                  CountVectorizer(decode_error='ignore',
                                                  ngram_range=(2, 2)))])),
                ('classifier', ComplementNB())])

In [14]:

# Fit the model
model_lm.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 Pipeline(steps=[('bow',
                                  CountVectorizer(decode_error='ignore',
                                                  ngram_range=(2, 2)))])),
                ('classifier', ComplementNB())])

In [15]:



# Predict on test data
y_pred = model_lm.predict(X_test)

# Performance of the model
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.972488038277512
